In [72]:
import requests
import time
import re
import datetime
from datetime import timedelta
from obspy import UTCDateTime
from obspy.core.event import read_events

class ISC_data():
    def __init__(self):
        self.EVENTID = []
        self.TYPE = []
        self.REPORTER = []
        self.STA = []
        self.LAT_st = []
        self.LON_st = []
        self.ELEV_st = []
        self.CHN = []
        self.DIST = []
        self.BAZ = []
        self.ISCPHASE = []
        self.REPPHASE = []
        self.DATE_st = []
        self.TIME_st = []
        self.RES = []
        self.TDEF = []
        self.AMPLITUDE = []
        self.PER = []
        self.AUTHOR = []
        self.DATE_ev = []
        self.TIME_ev = []
        self.LAT_ev = []
        self.LON_ev = []
        self.DEPTH_ev = []
        self.AUTHOR_ev = []
        self.TYPE_mag = []
        self.MAG = []
        



class ISCTTime():

    def __init__(self, lat1=33, lon1=33, lat2=43, lon2=43) -> None:
        # high quality data
        # self.baseurl =  'http://www.isc.ac.uk/cgi-bin/web-db-run?iscreview=&out_format=CSV&ttime=on&ttres=on&tdef=on&phaselist=&sta_list=&stn_ctr_lat=&stn_ctr_lon=&stn_radius=&max_stn_dist_units=deg&stnsearch=RECT&stn_top_lat=43&stn_bot_lat=33&stn_left_lon=33&stn_right_lon=43&stn_srn=&stn_grn=&searchshape=RECT&bot_lat=33&top_lat=43&left_lon=33&right_lon=43&ctr_lat=&ctr_lon=&radius=&max_dist_units=deg&srn=&grn=&start_year=2000&start_month=01&start_day=01&start_time=00%3A00%3A00&end_year=2000&end_month=2&end_day=01&end_time=00%3A00%3A00&min_dep=1&max_dep=40&min_mag=1&max_mag=&req_mag_type=Any&req_mag_agcy=Any&include_links=on&request=STNARRIVALS'
        # normal quality data
        self.baseurl =  'http://www.isc.ac.uk/cgi-bin/web-db-run?iscreview=&out_format=CSV&ttime=on&phaselist=&sta_list=&stn_ctr_lat=&stn_ctr_lon=&stn_radius=&max_stn_dist_units=deg&stnsearch=RECT&stn_top_lat=43&stn_bot_lat=33&stn_left_lon=33&stn_right_lon=43&stn_srn=&stn_grn=&searchshape=RECT&bot_lat=33&top_lat=43&left_lon=33&right_lon=43&ctr_lat=&ctr_lon=&radius=&max_dist_units=deg&srn=&grn=&start_year=2000&start_month=01&start_day=01&start_time=00%3A00%3A00&end_year=2000&end_month=2&end_day=01&end_time=00%3A00%3A00&min_dep=1&max_dep=40&min_mag=1&max_mag=&req_mag_type=Any&req_mag_agcy=Any&include_links=on&request=STNARRIVALS'
                         
        self.replace(
            bot_lat=lat1, top_lat=lat2, left_lon=lon1, right_lon=lon2,
            stn_top_lat=lat1, stn_bot_lat=lat2, stn_left_lon=lon1, stn_right_lon=lon2
        )

        self.data = ISC_data()

    def replace(self, **kwargs):
        for key in kwargs:
            value = kwargs[key]
            self.baseurl = re.sub(r'({})=(.+?)&'.format(key), r'\g<1>={}&'.format(value), self.baseurl)


    def replace_time(self, starttime, endtime):
        args ={
            'start_year': starttime.strftime('%Y'),
            'start_month': starttime.strftime('%m'),
            'start_day': starttime.strftime('%d'),
            'start_time': starttime.strftime('%H%%3A%M%%3A%S'),
            'end_year': endtime.strftime('%Y'),
            'end_month': endtime.strftime('%m'),
            'end_day': endtime.strftime('%d'),
            'end_time': endtime.strftime('%H%%3A%M%%3A%S'),
        }
        self.replace(**args)



    def error_checker(self, text):
        error_msg = 'but your request cannot be processed at the present time.'
        if error_msg in text:
            return False
        else:
            return True

    def text_2_data(self, text):
        lines = text.split('\n')
        data_line = False
        for line in lines:
            if (data_line):     # data line
                info = line.split(',')
                if (len(info) < 2):     # data end
                    data_line = False
                
                # read data
                try:
                    # EVENTID
                    match = re.search(r'event_id=(\d+)', info[0])
                    if match:
                        EVENTID = match.group(1)
                    else:
                        continue
                    # TYPE
                    TYPE = info[1]
                    # REPORTER
                    match = re.search(r'agency=([^ ]+)', info[2])
                    if match:
                        REPORTER = match.group(1)
                    else:
                        continue
                    # STA 
                    match = re.search(r'stacode=([^ ]+)', info[3])
                    if match:
                        STA = match.group(1)
                    else:
                        continue
                    # LAT_st 
                    LAT_st = float(info[4])
                    # LON_st 
                    LON_st = float(info[5])
                    # ELEV_st 
                    ELEV_st = float(info[6])
                    # CHN 
                    CHN = info[7]
                    # DIST 
                    DIST = float(info[8])
                    # BAZ 
                    BAZ = float(info[9])
                    # ISCPHASE 
                    ISCPHASE = info[10]
                    # REPPHASE 
                    REPPHASE = info[11]
                    # DATE_st 
                    DATE_st = info[12]
                    # TIME_st 
                    TIME_st = info[13]
                    # RES 
                    RES = info[14]
                    # TDEF 
                    TDEF = info[15]
                    # AMPLITUDE 
                    AMPLITUDE = info[16]
                    # PER 
                    PER = info[17]
                    # AUTHOR 
                    match = re.search(r'agency=([^ ]+)', info[18])
                    if match:
                        AUTHOR = match.group(1)
                    else:
                        continue
                    # DATE_ev 
                    DATE_ev = info[19]
                    # TIME_ev 
                    TIME_ev = info[20]
                    # LAT_ev 
                    LAT_ev = float(info[21])
                    # LON_ev 
                    LON_ev = float(info[22])
                    # DEPTH_ev 
                    DEPTH_ev = float(info[23])
                    # AUTHOR_ev 
                    match = re.search(r'agency=([^ ]+)', info[24])
                    if match:
                        AUTHOR_ev = match.group(1)
                    else:
                        continue
                    # TYPE_mag 
                    TYPE_mag = info[25]
                    # MAG 
                    MAG = float(info[26])

                    # add info
                    self.data.EVENTID.append(EVENTID)
                    self.data.TYPE.append(TYPE) 
                    self.data.REPORTER.append(REPORTER)
                    self.data.STA.append(STA)
                    self.data.LAT_st.append(LAT_st)
                    self.data.LON_st.append(LON_st)
                    self.data.ELEV_st.append(ELEV_st) 
                    self.data.CHN.append(CHN)
                    self.data.DIST.append(DIST)
                    self.data.BAZ.append(BAZ) 
                    self.data.ISCPHASE.append(ISCPHASE)
                    self.data.REPPHASE.append(REPPHASE) 
                    self.data.DATE_st.append(DATE_st) 
                    self.data.TIME_st.append(TIME_st)
                    self.data.RES.append(RES)
                    self.data.TDEF.append(TDEF) 
                    self.data.AMPLITUDE.append(AMPLITUDE) 
                    self.data.PER.append(PER)
                    self.data.AUTHOR.append(AUTHOR)
                    self.data.DATE_ev.append(DATE_ev)
                    self.data.TIME_ev.append(TIME_ev)
                    self.data.LAT_ev.append(LAT_ev)
                    self.data.LON_ev.append(LON_ev)
                    self.data.DEPTH_ev.append(DEPTH_ev)
                    self.data.AUTHOR_ev.append(AUTHOR_ev)
                    self.data.TYPE_mag.append(TYPE_mag)
                    self.data.MAG.append(MAG)
                except:
                    continue
                
                    
            if ('EVENTID  ,TYPE,REPORTER' in line):
                data_line = True


    def get_data(self, starttime = datetime.datetime(2000,1,1,0,0,0), endtime = datetime.datetime(2000,2,1,0,0,0)):
        time1 = starttime
        time2 = starttime + timedelta(days = 30)
        while True:     # loop time

            while True:  # continue trying to apply data
                self.replace_time(time1, time2)
                data_text = requests.get(self.baseurl)

                # check data
                if self.error_checker(data_text):
                    print('success in applying data, time: ',time1," - ",time2)
                    
                    break;    
                    # useful data
                else:
                    # failure, sleep and try again
                    print('fail in applying data, time: ',time1," - ",time2, "sleep 20 seconds and try again")
                    time.sleep(20)

            self.text_2_data(data_text.text)

            if (time2 > endtime):
                break
            else:
                time1 += timedelta(days = 30)
                time2 += timedelta(days = 30)
            time.sleep(2)


    def data_2_files(self,fname_st,fname_ev,fname_data):
        doc_st = open(fname_st,'w')
        doc_ev = open(fname_ev,'w')
        doc_data = open(fname_data,'w')

        evlist = {}
        stlist = {}
        data_info_ev = {}
        name_ev = ""
        for i in range(len(self.data.EVENTID)):
            old_name = name_ev
            name_ev = self.data.EVENTID[i]
            if (not name_ev in evlist): # new earthquake
                ortime = UTCDateTime(self.data.DATE_ev[i]+self.data.TIME_ev[i])
                lat_ev = self.data.LAT_ev[i]
                lon_ev = self.data.LON_ev[i]
                dep_ev = self.data.DEPTH_ev[i]
                id_ev = len(evlist)
                mag = self.data.MAG[i]
                evlist[name_ev] = 1
                doc_ev.write("%7d %s %s %9.4f %9.4f %9.4f %5.2f \n"%(id_ev,name_ev,ortime,lat_ev,lon_ev,dep_ev,mag))

            name_st = self.data.STA[i]
            if (not name_st in evlist): # new station
                lat_st = self.data.LAT_st[i]
                lon_st = self.data.LON_st[i]
                ele_st = self.data.ELEV_st[i]
                stlist[name_st] = 1
                doc_st.write("%6s %9.4f %9.4f %10.4f \n"%(name_st,lat_st,lon_st,ele_st))


            if (not name_ev in data_info_ev):   # a new event
                if (not i == 0 and len(data_info) > 0):    
                    doc_data.write('%7d %6d %2d %2d %2d %2d %5.2f %9.4f %9.4f %9.4f %5.2f %7d %s \n'%\
                                   (data_info_ev[old_name][0],data_info_ev[old_name][1].year,data_info_ev[old_name][1].month,data_info_ev[old_name][1].day,\
                                    data_info_ev[old_name][1].hour,data_info_ev[old_name][1].minute,data_info_ev[old_name][1].second+data_info_ev[old_name][1].microsecond/1000000,\
                                    data_info_ev[old_name][2],data_info_ev[old_name][3],data_info_ev[old_name][4],data_info_ev[old_name][5],ndata,data_info_ev[old_name][6]))
                    for key in data_info:
                        doc_data.write('%7d %7d %6s %9.4f %9.4f %9.4f %s %8.3f \n'%\
                                       (id_ev,data_info[key][0],data_info[key][1],data_info[key][2],\
                                        data_info[key][3],data_info[key][4],data_info[key][5],data_info[key][6]))
                ortime = UTCDateTime(self.data.DATE_ev[i]+self.data.TIME_ev[i])
                lat_ev = self.data.LAT_ev[i]
                lon_ev = self.data.LON_ev[i]
                dep_ev = self.data.DEPTH_ev[i]
                id_ev = len(data_info_ev)
                mag = self.data.MAG[i]
                data_info_ev[name_ev] = [id_ev,ortime,lat_ev,lon_ev,dep_ev,mag,name_ev]

                ndata = 0
                data_info = {}

                
                phase = self.data.ISCPHASE[i]+'_'+self.data.REPPHASE[i]
                if(phase.__contains__('P')):
                    if(self.data.ISCPHASE[i].__contains__('P')):
                        phase = self.data.ISCPHASE[i]
                    else:
                        phase = self.data.REPPHASE[i]
                    name_st = self.data.STA[i]
                    lat_st = self.data.LAT_st[i]
                    lon_st = self.data.LON_st[i]
                    ele_st = self.data.ELEV_st[i]
                    arrtime = UTCDateTime(self.data.DATE_st[i]+self.data.TIME_st[i])
                    if (name_st in data_info):      # if this station has been recorded
                        if (phase == self.data.ISCPHASE[i]):    # if the data is collected by ISC
                            data_info[name_st] = [ndata,name_st,lat_st,lon_st,ele_st,phase,arrtime-ortime]
                    else:      # not recorded
                        data_info[name_st] = [ndata,name_st,lat_st,lon_st,ele_st,phase,arrtime-ortime]
                        ndata += 1

            else:
                phase = self.data.ISCPHASE[i]+'_'+self.data.REPPHASE[i]
                if(phase.__contains__('P')):
                    if(self.data.ISCPHASE[i].__contains__('P')):
                        phase = self.data.ISCPHASE[i]
                    else:
                        phase = self.data.REPPHASE[i]
                    name_st = self.data.STA[i]
                    lat_st = self.data.LAT_st[i]
                    lon_st = self.data.LON_st[i]
                    ele_st = self.data.ELEV_st[i]
                    arrtime = UTCDateTime(self.data.DATE_st[i]+self.data.TIME_st[i])
                    if (name_st in data_info):      # if this station has been recorded
                        if (phase == self.data.ISCPHASE[i]):    # if the data is collected by ISC
                            data_info[name_st] = [ndata,name_st,lat_st,lon_st,ele_st,phase,arrtime-ortime]
                    else:      # not recorded
                        data_info[name_st] = [ndata,name_st,lat_st,lon_st,ele_st,phase,arrtime-ortime]
                        ndata += 1

            if(i == len(self.data.EVENTID)-1 and len(data_info) > 0):
                doc_data.write('%7d %6d %2d %2d %2d %2d %5.2f %9.4f %9.4f %9.4f %5.2f %7d %s \n'%\
                                (id_ev,ortime.year,ortime.month,ortime.day,ortime.hour,ortime.minute,ortime.second+ortime.microsecond/1000000,\
                                lat_ev,lon_ev,dep_ev,mag,ndata,name_ev))
                for key in data_info:
                    doc_data.write('%7d %7d %6s %9.4f %9.4f %9.4f %s %8.3f \n'%\
                                    (id_ev,data_info[key][0],data_info[key][1],data_info[key][2],\
                                     data_info[key][3],data_info[key][4],data_info[key][5],data_info[key][6]))


        doc_st.close()
        doc_ev.close()
        doc_data.close()


In [73]:
starttime = datetime.datetime(2000,1,1,0,0,0)
endtime = datetime.datetime(2023,3,1,0,0,0)

# print(starttime.second)


# p = UTCDateTime("2000-01-02"+"20:28:37.27")
# q = UTCDateTime("2000-01-02"+"21:28:37.27")

# print(q.second + q.microsecond/1000000)

isc_tt = ISCTTime( lat1=33, lon1=33, lat2=43, lon2=43)
isc_tt.get_data(starttime,endtime)
isc_tt.data_2_files("processed/st_list_test","processed/ev_list_test","processed/src_rec.dat_test")





success in applying data, time:  2000-01-01 00:00:00  -  2000-01-31 00:00:00
success in applying data, time:  2000-01-31 00:00:00  -  2000-03-01 00:00:00
success in applying data, time:  2000-03-01 00:00:00  -  2000-03-31 00:00:00


In [74]:
# isc_tt.data_2_files("processed/st_list","processed/ev_list","processed/src_rec.dat")


In [75]:
a = {}
a[1] = 1
print(len(a))

1
